In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from datasets import Dataset

/home/rizal/Documents/src/LLM/fine_tunning/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id = "deepseek-ai/deepseek-coder-1.3b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

bnb = BitsAndBytesConfig(
    load_in_8bit=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    dtype="auto",
    quantization_config=bnb
)


In [ ]:
lora_config = LoraConfig(
    r=16,                 # rank kecil → hemat VRAM
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [4]:
model = get_peft_model(model, lora_config)

In [5]:
from prompt_generator import CodeReviewPromptGenerator
prompt_generator = CodeReviewPromptGenerator()

def format_prompt(data):
    prompt = prompt_generator.generate_style_review_prompt(
        added_code=data["added_code"],
        deleted_code=data["deleted_code"],
        full_function_code=data["full_function_code"],
        function_name=data["function_name"],
    )
    target = data["code_review_suggestion"]
    return prompt, target

In [6]:
def tokenize_example(example, max_length=512):  # sequence lebih pendek → hemat memori
    prompt, target = format_prompt(example)
    full_text = prompt + target
    tokens = tokenizer(
        full_text,
        truncation=True,
        max_length=max_length,
        padding="max_length"
    )
    return tokens

In [ ]:
import json

dataset = []
with open("merged_dataset.json", "r") as f:
    dataset = json.load(f)

# only for testing, use all dataset for actual training
dataset = dataset[:30]
tokenized_data = [tokenize_example(e) for e in dataset]

In [8]:
dataset_hf = Dataset.from_list([{k: v for k, v in t.items()} for t in tokenized_data])
train_size = int(0.8 * len(dataset_hf))
train_dataset = dataset_hf.select(range(train_size))
test_dataset = dataset_hf.select(range(train_size, len(dataset_hf)))

def add_labels(batch):
    batch["labels"] = batch["input_ids"]
    return batch

train_dataset = train_dataset.map(add_labels)
test_dataset = test_dataset.map(add_labels)

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

Map: 100%|██████████| 6/6 [00:00<00:00, 888.50 examples/s]


In [ ]:
training_args = TrainingArguments(
    output_dir="./lora_deepseek_coder_1.3b_style_review",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="epoch",
    learning_rate=2e-4,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    logging_steps=10,
    fp16=True,
    push_to_hub=False,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)


In [ ]:
trainer.train()

In [ ]:
model.save_pretrained("./lora_deepseek_coder_1.3b_style_review")

In [ ]:
metrics = trainer.evaluate()
print(metrics)

In [ ]:
# -------------------------------
# 9️⃣ Generate output contoh
# -------------------------------
sample = dataset[0]
prompt, _ = format_prompt(sample)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=512)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/home/rizal/Documents/src/LLM/fine_tunning/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


You are a code reviewer. Analyze this Python code change and respond EXACTLY in the format below.

Full function `handle_api_response`:
```python
def handle_api_response(api_response):
    if api_response is not None:
        if isinstance(api_response, dict):
            if "status_code" in api_response:
                status_code = api_response["status_code"]
                if status_code == 200:
                    response_data = api_response["data"]
                    # Validate response data
                    if isinstance(response_data, list):
                        for item in response_data:
                            if not isinstance(item, dict):
                                raise ValueError("Invalid response data")
                    elif not isinstance(response_data, dict):
                        raise ValueError("Invalid response data")
                    return response_data
                else:
                    raise ValueError("Invalid status code")
   